In [1]:
import pandas as pd
import numpy as np
import pickle 

import math

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.preprocessing import MinMaxScaler

In [2]:
df = pd.read_csv(r"C:\Users\mailt\Desktop\Github Projects\Recommendation-Systems\datasets\processed_df.csv")
df10 = df[["originalTitle","genres","averageRating"]]

In [3]:
df.startYear = [math.ceil((x%2000)/5) for x in df.startYear]
df.numVotes = [int(x/5000) for x in df.numVotes]

In [4]:
for i in range(df.shape[0]):
    if df.runtimeMinutes[i]<=30:
        df.runtimeMinutes[i] = "short_runtime"
    elif df.runtimeMinutes[i]>30 and df.runtimeMinutes[i]<=60:
        df.runtimeMinutes[i] = "normal_runtime"
    elif df.runtimeMinutes[i]>60:   
        df.runtimeMinutes[i]="long_runtime"

<ipython-input-4-5a712898dfcb>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.runtimeMinutes[i] = "normal_runtime"
C:\Users\mailt\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
<ipython-input-4-5a712898dfcb>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.runtimeMinutes[i]="long_runtime"
<ipython-input-4-5a712898dfcb>:3: S

In [5]:
v = CountVectorizer()
x = v.fit_transform(df["runtimeMinutes"])
df1 = pd.DataFrame(x.toarray(), columns=v.get_feature_names())
df = pd.concat([df, df1], axis=1)
df.drop(["runtimeMinutes"], axis = 1, inplace=True) 

In [6]:
scaler = MinMaxScaler()
df[['startYear', 'averageRating', 'numVotes']] = scaler.fit_transform(df[['startYear', 'averageRating', 'numVotes']])

In [7]:
df.head()

,tconst,originalTitle,startYear,genres,averageRating,numVotes,long_runtime,normal_runtime,short_runtime
0,tt2301451,Ozymandias,0.6,"Crime,Drama,Thriller",1.00,0.075269,0,1,0
1,tt4283088,Battle of the Bastards,0.8,"Action,Adventure,Drama",0.98,0.104839,0,1,0
2,tt4283094,The Winds of Winter,0.8,"Action,Adventure,Drama",0.98,0.072581,1,0,0
3,tt6723680,Senshi,0.8,"Action,Adventure,Animation",0.98,0.008065,0,0,1
4,tt1833197,"Goodbye, Michael",0.6,Comedy,0.96,0.005376,0,1,0


In [8]:
title = df[["originalTitle","genres"]]

In [9]:
df.genres = [x.replace(","," ") for x in df.genres]

In [10]:
df.genres.str.split(expand=True).stack().value_counts()

Drama          3506
Comedy         2860
Crime          1412
Action         1270
Animation      1109
Adventure       998
Mystery         876
Romance         734
Fantasy         507
Documentary     443
Thriller        407
Reality-TV      367
Family          309
Sci-Fi          288
Horror          238
History         164
Music           163
Game-Show       125
Sport           120
Biography       103
Talk-Show       101
News             47
War              35
Musical          30
Short            18
Western          13
Adult             1
dtype: int64

In [11]:
v = CountVectorizer()
x = v.fit_transform(df["genres"])
df1 = pd.DataFrame(x.toarray(), columns=v.get_feature_names())
df = pd.concat([df, df1], axis=1)
df.drop(["genres","tconst","adult", "western","short","musical","news","originalTitle"], axis = 1, inplace=True) 

In [12]:
# df.drop(columns=["adult", "western","short","musical","news","originalTitle"], inplace=True)

In [13]:
df.head()

,startYear,averageRating,numVotes,long_runtime,normal_runtime,short_runtime,action,adventure,animation,biography,...,mystery,reality,romance,sci,show,sport,talk,thriller,tv,war
0,0.6,1.00,0.075269,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0.8,0.98,0.104839,0,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0.8,0.98,0.072581,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.8,0.98,0.008065,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0.6,0.96,0.005376,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
import scipy
mat = scipy.sparse.csr_matrix(df1.values)
from sklearn.metrics.pairwise import sigmoid_kernel
sig1 = sigmoid_kernel(mat,mat)

In [15]:
index = pd.Series(df10.index, index = df10.originalTitle).drop_duplicates()

In [16]:
def recommend(title, sig=sig1):
    idx = index[title]
    scores = list(enumerate(sig1[idx]))
    scores = sorted(scores, key=lambda x:x[1], reverse=True)
    scores= scores[1:11]
    mov_index = [i[0] for i in scores]
    return df10.iloc[mov_index]

In [18]:
recommend("Fargo")

,originalTitle,genres,averageRating
47,Koombiyo,"Crime,Drama,Thriller",950
67,Breaking Bad,"Crime,Drama,Thriller",940
95,The Wire,"Crime,Drama,Thriller",930
136,The Toll,"Crime,Drama,Thriller",919
182,Pilot,"Crime,Drama,Thriller",910
218,High,"Crime,Drama,Thriller",900
257,Fargo,"Crime,Drama,Thriller",890
312,Senke nad Balkanom,"Crime,Drama,Thriller",890
375,Umbre,"Crime,Drama,Thriller",880
481,The Shield,"Crime,Drama,Thriller",869
